# Fine-Tuning Mistral 7b for Finance

In [1]:
from huggingface_hub import snapshot_download
from pathlib import Path
import torch
from transformers import AutoModel, AutoModelForCausalLM, AutoModelForMaskedLM, MistralForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

2024-09-23 10:15:58.261357: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)

In [9]:

snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3", allow_patterns=["*.json", "*.bin", "consolidated.safetensors", "tokenizer.model.v3"], local_dir=mistral_models_path)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

'/Users/johnmoses/mistral_models/7B-Instruct-v0.3'

In [ ]:
model = MistralForCausalLM.from_pretrained(
    mistral_models_path,
    # use_safetensors=True,
    )

In [14]:
base_model_id = "mistralai/Mistral-7B-Instruct-v0.3"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    # bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# set pad_token_id equal to the eos_token_id if not set
# if tokenizer.pad_token_id is None:
#   tokenizer.pad_token_id = tokenizer.eos_token_id

# Set reasonable default for models without max length
# if tokenizer.model_max_length > 100_000:
#   tokenizer.model_max_length = 2048

# Set chat template
# DEFAULT_CHAT_TEMPLATE = "{% for message in messages %}\n{% if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{% elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{% endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}"
# tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE

In [ ]:
model = AutoModelForCausalLM.from_pretrained(base_model_id)

In [1]:
import torch
# Check if MPS is available
print(torch.backends.mps.is_available())
# Check if built with MPS.
print(torch.backends.mps.is_built())

True
True
